In [89]:
import numpy as np
import pandas as pd
from scipy.stats import zscore, chi2
from scipy.spatial.distance import mahalanobis
from sklearn.neighbors import LocalOutlierFactor
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import STL
import ruptures as rpt
import pmdarima as pm
from sklearn.decomposition import PCA


### Read files

In [3]:
files = {
    "Current_Assets": "../../data/anomaly/Ordered_Current_Assets_PRD_Data.csv",
    "Current_Liabilities": "../../data/anomaly/Ordered_Current_Liabilities_PRD_Data__5300177_.csv",
    "Profit_Before_Tax": "../../data/anomaly/Ordered_Profit_Before_Tax_PRD_Data__5300258_.csv",
    "Revenue": "../../data/anomaly/Ordered_Revenue__5300260__PRD_Data.csv",
    "Total_Expenses": "../../data/anomaly/Ordered_Total_Expenses_PRD_Data__5300002_.csv"
}

dfs = []
for name, path in files.items():
    df = pd.read_csv(path)
    df = df.rename(columns={"value": name})
    dfs.append(df)

# Merge on period_id
data = dfs[0]
for df in dfs[1:]:
    data = data.merge(df, on="period_id")

# Set period_id as index
data = data.set_index("period_id")

In [9]:
data.head(20)

,Current_Assets,Current_Liabilities,Profit_Before_Tax,Revenue,Total_Expenses
period_id,,,,,
2023_M_PRD_3,1.132491e+10,1.158456e+10,1.019800e+10,2.180767e+10,1.209075e+10
2023_M_PRD_6,1.105986e+10,1.292136e+10,2.523593e+09,6.072411e+09,3.720507e+09
2023_M_PRD_9,1.737128e+10,2.669838e+10,4.498083e+09,1.238945e+10,8.145085e+09
2023_M_PRD_12,1.397483e+10,1.658028e+10,6.676332e+09,1.923193e+10,1.291804e+10
2024_M_PRD_1,1.155809e+10,9.878206e+09,3.440362e+10,2.192677e+10,1.199577e+10
2024_M_PRD_3,1.287243e+10,1.346934e+10,1.052172e+10,2.600736e+10,1.665185e+10
2024_M_PRD_5,1.552990e+10,9.564117e+09,2.414810e+09,4.560325e+09,2.363952e+09
2024_M_PRD_6,2.371369e+10,1.079530e+10,4.227699e+09,6.790236e+09,2.858876e+09
2024_M_PRD_7,1.482473e+10,1.125222e+10,5.393750e+09,9.110274e+09,4.122146e+09


### Detect if stationary v/s non-stationary

In [61]:
def is_stationary(series, alpha=0.05):
    """Robust stationarity test using both ADF and KPSS."""
    series = series.dropna()

    # --- ADF test ---
    try:
        adf_p = adfuller(series, autolag="AIC")[1]
        adf_stationary = adf_p < alpha
    except:
        adf_stationary = True  # fallback

    # --- KPSS test ---
    try:
        kpss_p = kpss(series, regression="c", nlags="auto")[1]
        kpss_stationary = kpss_p > alpha
    except:
        kpss_stationary = adf_stationary  # fallback if fails

    # --- Decision rule ---
    if adf_stationary and kpss_stationary:
        return True   # confident stationary
    else:
        return False  # treat as non-stationary


### Anomaly Detection

In [62]:
def stationary_anomalies(data):

    zscore_anomalies = pd.DataFrame(0, index=data.index, columns=data.columns)
    mah_anomalies = pd.DataFrame(0, index=data.index, columns=data.columns)
    lof_anomalies = pd.DataFrame(0, index=data.index, columns=data.columns)

    for col in data.columns:
        col_values = data[col].values.reshape(-1, 1)
        
        # Z-Score method
        z_scores = np.abs(zscore(col_values, nan_policy="omit"))
        zscore_anomalies[col] = (z_scores > 2).astype(int).flatten()
        
        # LOF method
        if len(col_values) > 5:  # need enough points
            lof = LocalOutlierFactor(n_neighbors=5, contamination=0.1)
            lof_pred = lof.fit_predict(col_values)
            lof_anomalies[col] = (lof_pred == -1).astype(int)
        else:
            lof_anomalies[col] = 0  # fallback if too few points
        
    mean_vec = data.mean(axis=0).values
    cov_matrix = np.cov(data.values, rowvar=False)
    inv_cov_matrix = np.linalg.inv(cov_matrix)

    mahal_distances = data.apply(
        lambda row: mahalanobis(row, mean_vec, inv_cov_matrix), axis=1
    )
    threshold = chi2.ppf(0.8, data.shape[1])  # chi-square cutoff
    mah_flags = (mahal_distances ** 2 > threshold).astype(int)

    # Expand to all columns (if period flagged, mark all its columns)
    mah_anomalies = pd.DataFrame(
        np.repeat(mah_flags.values[:, None], data.shape[1], axis=1),
        index=data.index,
        columns=data.columns
    )
    anomalies_df = lof_anomalies + zscore_anomalies + mah_anomalies
    final_anomalies = (anomalies_df >= 2).astype(int)
    return final_anomalies

### Non-stationary Anomaly

In [90]:
def nonstationary_univariate_anomalies(series, period=2):
    # Use STL decomposition
    series = series.dropna()
    stl = STL(series, period=period, robust=True)
    result = stl.fit()
    resid = result.resid

    ##  --- Method 1:Robust z-score using MAD ---
    mad = np.median(np.abs(resid - np.median(resid)))
    robust_z = np.abs((resid - np.median(resid)) / (mad if mad > 0 else 1))
    stl_flags = (robust_z > 3.5).astype(int)

    # --- Method 2: Change point detection (on trend component) ---
    algo = rpt.Pelt(model="rbf").fit(result.trend.values)
    change_points = algo.predict(pen=5)  # penalization controls sensitivity
    cp_flags = pd.Series(0, index=series.index)
    for cp in change_points[:-1]:  # exclude last point (always marked)
        cp_flags.iloc[cp:] = 1  # mark all points after change

    # Method 3: Fit ARIMA automatically
    model = pm.auto_arima(series, seasonal=False, error_action="ignore", suppress_warnings=True)
    preds = model.predict_in_sample()

    # Compute residuals
    residuals = series - preds

    # Use MAD-based robust z-score
    mad = np.median(np.abs(residuals - np.median(residuals)))
    robust_z = np.abs((residuals - np.median(residuals)) / (mad if mad > 0 else 1))
    arima_flags = (robust_z > 3.5).astype(int)

    # --- Combine (union of STL residual + change points) ---
    combined_detection = (stl_flags + cp_flags + arima_flags)
    final_flags = (combined_detection >= 2).astype(int)
    

    return final_flags, residuals

In [ ]:
def nonstationary_multivariate_anomalies(residuals_df, variance_threshold=0.9):
    """Run PCA + Mahalanobis on residuals to detect joint anomalies."""

    X = residuals_df.values

    ### Method 1 - PCA ######
    # --- PCA fit ---
    pca = PCA(n_components=variance_threshold)
    X_pca = pca.fit_transform(X)
    X_reconstructed = pca.inverse_transform(X_pca)

    # --- Cell-wise reconstruction error ---
    cell_errors = (X - X_reconstructed) ** 2
    error_df = pd.DataFrame(cell_errors, index=residuals_df.index, columns=residuals_df.columns)

    # --- Column-wise thresholds (mean + 3*std) ---
    thresholds = error_df.mean(axis=0) + 3 * error_df.std(axis=0)
    pca_flags = (error_df > thresholds).astype(int)

    ### Method 2 - Mahalanobis on Residuals ######
    # --- Mahalanobis Distance on Residuals ---
    mean_vec = residuals_df.mean(axis=0).values
    cov_matrix = np.cov(X, rowvar=False)
    inv_cov = np.linalg.pinv(cov_matrix)

    # Centered data
    diffs = X - mean_vec

    # Contribution matrix: elementwise product of (diffs) and (inv_cov @ diffs^T)
    contribs = diffs @ inv_cov * diffs  # shape (n_samples, n_features)
    contribs_df = pd.DataFrame(contribs, index=residuals_df.index, columns=residuals_df.columns)

    # Threshold per column: mean + 3*std (can also use chi-square decomposition)
    thresholds = contribs_df.mean(axis=0) + 3 * contribs_df.std(axis=0)
    mahal_flags = (contribs_df > thresholds).astype(int)

    combined_detection = (pca_flags + mahal_flags)
    final_flags = (combined_detection >= 1).astype(int)

    return final_flags


In [105]:
final_anomalies = pd.DataFrame(0, index=data.index, columns=data.columns)
residuals = pd.DataFrame(index=data.index, columns=data.columns)

for col in data.columns:
    series = data[col]
    if is_stationary(series):
        #print('Stationary ', col)
        # For stationary attributes, include them in multivariate detection
        subset = data[[col]]
        flags = stationary_anomalies(subset)
        final_anomalies[col] = flags[col]
    else:
        #print('Non-stationary', col)
        # For non-stationary univariate test
        final_anomalies[col], resid = nonstationary_univariate_anomalies(series)
        residuals[col] = resid

        # Multivariate time-series check
        residuals = residuals.fillna(0)
        global_flags = nonstationary_multivariate_anomalies(residuals)

        # combine univariate & multivariate
        for col in final_anomalies.columns:
            final_anomalies[col] = np.maximum(final_anomalies[col], global_flags[col])

/opt/miniconda3/envs/bsq_aws/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/envs/bsq_aws/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/envs/bsq_aws/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/envs/bsq_aws/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/envs/bsq_aws/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure

In [106]:
final_anomalies.head(30)

,Current_Assets,Current_Liabilities,Profit_Before_Tax,Revenue,Total_Expenses
period_id,,,,,
2023_M_PRD_3,0,0,0,0,0
2023_M_PRD_6,0,0,0,1,0
2023_M_PRD_9,0,0,1,0,0
2023_M_PRD_12,0,0,0,0,0
2024_M_PRD_1,0,1,1,0,0
2024_M_PRD_3,0,0,0,0,0
2024_M_PRD_5,0,0,0,0,0
2024_M_PRD_6,0,0,0,0,0
2024_M_PRD_7,1,0,0,0,0
